# Q-Learning with Battery Example
Purpose of this notebook is to demonstrate the ability of a reinforcement learning agent based on Q-Learning to learn to control a battery.

In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from energy_py import Runner
from energy_py.agents import DQN
from energy_py.envs import BatteryEnv

In [2]:
#  set random seeds for repeatability
np.random.seed(42)
tf.set_random_seed(42)

In [3]:
#  define a total number of steps for the experiment to run
TOTAL_STEPS = 100000

#  to setup the agent we use a dictionary
#  a dictionary allows us to eaisly save the config to csv if we want
agent_config = {'discount': 0.97,                 #  the discount rate
                'tau': 0.001,                     #  parameter that controls the copying of weights from online to target network
                'total_steps': TOTAL_STEPS,   
                'batch_size': 32,                 #  size of the minibatches used for learning
                'layers': (50, 50),               #  structure of the neural network used to approximate Q(s,a)
                'learning_rate': 0.0001,          #  controls the stength of weight updates during learning       
                'epsilon_decay_fraction': 0.3,    #  a fraction as % of total steps where epsilon decayed from 1.0 to 0.1
                'memory_fraction': 0.4,           #  the size of the replay memory as a % of total steps
                'memory_type': 'deque',           #  the replay memory implementation we want
                'process_observation': 'standardize',
                'process_target': 'normalize'}    #  default is to normalize each batch (i.e. scale by min/max)

#  keep all of the BatteryEnv variables (episode length, efficiency etc) at their defaults
#  we just need to let our env know where our state.csv and observation.csv are (data_path)
env = BatteryEnv(data_path=os.getcwd())

In [4]:
#  Runner is a class that helps us with experiments - tracking rewards, writing environment info to csv and managing TensorBoard
#  in this notebook we just use it to track rewards for us
runner = Runner()

#  initialize Tensorflow machinery
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #  add the tf session and the environment to the agent config dictionary
    #  and initialize the agent
    agent_config['sess'] = sess
    agent_config['env'] = env
    agent = DQN(**agent_config)
    
    #  initial values for the step and episode number
    step, episode = 0, 0

    #  outer while loop runs through multiple episodes
    while step < TOTAL_STEPS:
        episode += 1
        done = False
        observation = env.reset()
        
        #  inner while loop runs through a single episode
        while not done:
            step += 1
            #  select an action
            action = agent.act(observation)
            
            #  take one step through the environment
            next_observation, reward, done, info = env.step(action)
            
            #  store the experience
            agent.remember(observation, action, reward,
                           next_observation, done)
            
            #  moving to the next time step
            observation = next_observation
            #  saving the reward 
            runner.append(reward)
            
            #  we don't start learning until the memory is half full
            if step > int(agent.memory.size * 0.5):
                train_info = agent.learn()
        
        if episode % 200 == 0:
            print('progress - {:.1f}% - episode {:.0f} - run time {:.2f} - episode reward {:.2f} - avg rewards {:.2f}'.format(100*step/TOTAL_STEPS,
                                                                                                                              episode,
                                                                                                                              runner.calc_time(),
                                                                                                                              sum(runner.ep_rewards),
                                                                                                                              runner.avg_rew))
        #  the report method should be run at the end of each episode
        runner.report({'ep': episode,
                       'step': step})

progress - 9.4% - episode 200 - run time 0.14 - episode reward -0.04 - avg rewards -16.08
progress - 18.8% - episode 400 - run time 0.34 - episode reward -22.11 - avg rewards -27.08
progress - 28.2% - episode 600 - run time 2.71 - episode reward -10.66 - avg rewards -4.81
progress - 37.6% - episode 800 - run time 5.51 - episode reward 48.42 - avg rewards -4.10
progress - 47.0% - episode 1000 - run time 8.46 - episode reward 4.96 - avg rewards -24.96
progress - 56.4% - episode 1200 - run time 11.33 - episode reward -1.23 - avg rewards -5.22
progress - 65.8% - episode 1400 - run time 14.27 - episode reward -0.22 - avg rewards 9.98
progress - 75.2% - episode 1600 - run time 17.35 - episode reward -36.84 - avg rewards -4.87
progress - 84.6% - episode 1800 - run time 20.32 - episode reward 44.04 - avg rewards 14.32


KeyboardInterrupt: 

In [ ]:
#  energy_py uses TensorBoard for logging - for the scope of this notebook example we will do
#  some plotting using matplotlib

import matplotlib.pyplot as plt

episode_rewards = runner.global_rewards
plt.plot(episode_rewards)

In [ ]:
#  we can also look at what happened in our last episode
ep_hist = pd.DataFrame.from_dict(info)
ep_hist.head()

In [ ]:
plt.plot(ep_hist.loc[:, 'new_charge'])

In [ ]:
plt.plot(ep_hist.loc[:, 'electricity_price'])

In [ ]:
ep_hist.loc[:, 'reward'].sum()